In [1]:

import pandas as pd
import numpy as np
from scipy import signal, misc
from datetime import datetime, timedelta


import os
import json

In [7]:


def split_month(t):
    year,month,rest=str(t).split("-")
    return month

def split_day(t):
            day,hour=str(t).split(" ")
            return day
def split_year(t):
    year,month,rest=str(t).split("-")
    return year

def combine_mins(t):
            day,hour=str(t).split(" ")
            hour,mins,seconds=str(hour).split(":")
            return hour

def split_date(data):
    data['date']=data.timetableStamp.apply(split_day)
    data['year']=data.timetableStamp.apply(split_year)
    data['month']=data.timetableStamp.apply(split_month)
    data['hour']=data.timetableStamp.apply(combine_mins)
    return data



def milk_edgecase(milk_temp):
    if milk_temp==4: #below lower limit
        return 2
    elif int(milk_temp) in range(4,6): #safe temperature
        return 1
    elif int(milk_temp) in range(6,30): #fresh product or empty tank
        return 0
    else:
        return 4

    
def tss_edgecase(TssTemperature):
    if(TssTemperature>=0):
        return 1
    elif TssTemperature<= -4:
        return 2
    else:
        return 0



In [8]:
df = pd.read_csv('Bibi_sensor_data_2019_09_10.csv')
data=pd.DataFrame(df,columns=['timetableStamp','MilkTemperature','AgitatorStatus'])
data.head(10)

,timetableStamp,MilkTemperature,AgitatorStatus
0,2019-09-10 00:00:01,5.1,0
1,2019-09-10 00:02:01,5.2,0
2,2019-09-10 00:04:01,5.3,0
3,2019-09-10 00:06:01,5.3,0
4,2019-09-10 00:08:01,5.3,1
5,2019-09-10 00:10:01,4.6,0
6,2019-09-10 00:12:01,4.9,0
7,2019-09-10 00:14:01,5.2,0
8,2019-09-10 00:16:01,5.3,0
9,2019-09-10 00:18:01,5.4,0


In [9]:
#checking for acm mode
acm=[]
agitator_on=0
acm_on=0
flag=0
for row in data.iterrows():
    if row[1][1]<=6: #if milk is present and below upper limit
        if row[1][2]==1:
            agitator_on=agitator_on+1
            if agitator_on==1: #could be in acm or just on for long
                flag=1  
                starttime=row[1][0]
                startmilktemp=row[1][1]
                #print('Starting cycle at ',starttime,' with milk temp: ',startmilktemp)
                acm.append(1)
            
            elif acm_on!=0: #end of acm cylce
                acm.append(1)
                #print('Ending cycle at ',row[1][0],' with milk temp: ',row[1][1],' with ',acm_on,' cycles')
                flag=1
                acm_on=0
                agitator_on=0
            else:
                acm.append(0)
        elif row[1][2]==0:
            if flag==1: #cyclic mode has just started
                acm_on=acm_on+1 
                acm.append(1)
                #print('Agitator in cyclic mode for ',acm_on,' cycles')
            else:
                acm.append(0)
        else:
            acm.append(0)
    else:
            acm.append(0)
            
print(len(acm), len(data))            
data['acm']=acm
data.head(5)

720 720


,timetableStamp,MilkTemperature,AgitatorStatus,acm
0,2019-09-10 00:00:01,5.1,0,0
1,2019-09-10 00:02:01,5.2,0,0
2,2019-09-10 00:04:01,5.3,0,0
3,2019-09-10 00:06:01,5.3,0,0
4,2019-09-10 00:08:01,5.3,1,1


In [10]:
starttime=datetime.now()
starttemp=0
count=0
data = split_date(data)
data.head()

,timetableStamp,MilkTemperature,AgitatorStatus,acm,date,year,month,hour
0,2019-09-10 00:00:01,5.1,0,0,2019-09-10,2019,09,00
1,2019-09-10 00:02:01,5.2,0,0,2019-09-10,2019,09,00
2,2019-09-10 00:04:01,5.3,0,0,2019-09-10,2019,09,00
3,2019-09-10 00:06:01,5.3,0,0,2019-09-10,2019,09,00
4,2019-09-10 00:08:01,5.3,1,1,2019-09-10,2019,09,00


In [11]:

for row in data.iterrows():
    if row[1][1]<=6: #if milk temperature is below upper limit
        count=count+1
        if count==1:
            starttime=row[1][0]
            starttemp=row[1][1]
            print(row[1][0],': Agitator ON with milk temp ',row[1][1])
        
    if row[1][3]==1 & count!=0:
        
        if count!=1:
            print(row[1][0],': Agitator got into cyclic Mode, with count=',count,' and milk temp=',row[1][1])
            print('Time Taken to get into ACM ',(row[1][0]-starttime))
        count=0

2019-09-10 00:00:01 : Agitator ON with milk temp  5.1
2019-09-10 00:08:01 : Agitator got into cyclic Mode, with count= 5  and milk temp= 5.3


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [42]:
tsstemp1=0
tsstemp2=0
count1=0
count2=0
starttime=datetime.now()

for row in data.iterrows():
        if row[1][6]==0: #milk temp over 6, start count1
            count1=count1+1
            
            if count1==1:
                starttime1=row[1][1] #get start time
                milktemp1=row[1][0]

        elif row[1][6]==1: #milk temp in range, save count1, start count2
            count2=count2+1
            if count2==1:
                 endtime1=row[1][1] #get end time as soon as it reached safe temp
                  
                 if(count1>1):
                    print('-'*40,'\nCount1: ',count1+1)
                    print('Milk temp at start: ',milktemp1,' , Milk temp now: ',row[1][0])
                    print('Start time is: ',starttime1,'End time is: ',endtime1)
                 count1=0
                 starttime2=row[1][1]
                 milktemp2=row[1][0]
                 #FMT = '%Y-%M-%T %H:%M:%S'
                 #tdelta = datetime.strptime(endtime1, FMT) - datetime.strptime(starttime1, FMT)
                 
                 #tdelta = endtime1-starttime1
                 
                 #print('\tTime taken to reach upper limit: ',tdelta)
            

        elif row[1][6]==2:
            count3=count3+1
            count2=0
            if(count3==1):
                endtime2=row[1][1] #get end time as soon as it reached safe temp
                print('-'*40,'\nCount2: ',count2+1)
                
                print('Milk temp at start: ',milktemp2,' , Milk temp now: ',row[1][0])
                print('Start time is: ',starttime2,'End time is: ',endtime2)
                #tdelta = endtime2-starttime2
                #print('\tTime taken to reach Lower limit: ',tdelta)
            
              

---------------------------------------- 
Count2:  1
Milk temp at start:  5.1  , Milk temp now:  4.0
Start time is:  2019-09-10 00:00:01 End time is:  2019-09-10 01:40:01
---------------------------------------- 
Count1:  20
Milk temp at start:  7.4  , Milk temp now:  4.1
Start time is:  2019-09-10 08:28:05 End time is:  2019-09-10 09:40:05


In [ ]:
data = pd.read_sql("SELECT tss_temp_mean as tss_temp, timestamp FROM Promethean.device_sensor_data WHERE serial_number="+str(device),con=engine)
    data = split_date(data)
    year=[]
    month=[]
    tss_charging=[]
    try:

        year_data=data.groupby('year')
        for year,year_group in year_data:
            month_data=year_group.groupby('month')
            #print('Yearly:\n',year_group)
            for month,month_group in month_data:
                charging_time=[]
                #print('Monthly:\n',month_group)
                grouped=month_group.groupby('date')
                for date, group in grouped:
                    #print('Date:\n',group)
                    i=0
                    counter=0
                    tss_list=list(group.tss_temp)
                    for row in tss_list[:-1]:
                        if int(row) not in range(-6,-4):
                            counter=counter+1
                            if int(tss_list[i+1]) in range(-6,-4):
                                charging_time.append(counter)
                                counter=0
                                break
                        i=i+1
                print('Year: ',year,', Month: ',month,', Length of days:',len(month_group['date'].unique()))
                print('Average TSS charging time:',(sum(charging_time)/(len(month_group['date'].unique()))))
                df.loc[j]=(j,device,year,month,len(month_group['date'].unique()),(sum(charging_time)/(len(month_group['date'].unique()))))
                j=j+1